In [1]:
CFG = {
    "buying_order_n": 10,
    "selling_order_percentage":30,
    "sampling_size": 250,
}

In [14]:
import datetime as dt
import random

import numpy as np
import pandas as pd

import kquant as kq


# API_account
def set_api_account():
    kq.set_api("KRX2308020", "EQDkUcyI3dK6oIAXqAR8BXOK4bKxHHmH")
    return None


set_api_account()

In [3]:
class STATUS_LOADER:
    """
    STATUS_LOADER : 상태 정보 추출 클래스
    """

    def __init__(self, dict_df_result: dict, dict_df_position: dict) -> None:
        """
        STATUS_LOADER의 생성자

        :param dict dict_df_result: dict_df_result 입니다.
        :param dict dict_df_position: dict_df_position 입니다.
        :return: None
        :rtype: None
        """
        self.dict_df_result = dict_df_result
        self.dict_df_position = dict_df_position

    def get_current_cash(self) -> float:
        """
        현재 보유 cash를 반환하는 메서드

        :return : 현재 보유 cash
        :rtype: float
        """
        _dict_df_result = self.dict_df_result
        try:
            _df_result_total = _dict_df_result["TOTAL"]
            _current_cash = (
                _df_result_total.sort_values("DATE").tail(1)["CASH"].values[0]
            )
            return _current_cash
        except:
            return 1_000_000_000.0

    def get_status_df(self) -> pd.DataFrame:
        """
        현재 보유 position 관련 정보를 반환하는 메서드

        :return: 보유한 symbol, 갯수, 구매가격, 현재 가격을 데이터프레임 형태로 가져옵니다.
        :rtype: pd.DataFrame
        """
        current_symbol_list = list()
        _dict_df_result = self.dict_df_result
        _dict_df_position = self.dict_df_position

        _total_symbols = sorted(_dict_df_position.keys())

        for _symbol in _total_symbols:
            try:
                _symbol_result_df = _dict_df_result[_symbol]
                _symbol_position_df = _dict_df_position[_symbol]

                _current_price = (
                    _symbol_result_df.sort_values("DATE").tail(1)["PRICE"].values[0]
                )
                _trade_price = _symbol_position_df["TRADE_PRICE"].values[0]
                _current_qty = _symbol_position_df["QTY"].values[0]

                current_symbol_list.append(
                    {
                        "SYMBOL": _symbol,
                        "CURRENT_QTY": _current_qty,
                        "CURRENT_PRICE": _current_price,
                        "TRADE_PRICE": _trade_price,
                    }
                )
            except:
                pass
        return pd.DataFrame(
            current_symbol_list,
            columns=["SYMBOL", "CURRENT_QTY", "CURRENT_PRICE", "TRADE_PRICE"],
        )

In [4]:
class SYMBOL_LOADER:
    """
    SYMBOL_LOADER : 거래가능한 주식 symbol을 필터-추출하는 클래스
    """

    @staticmethod
    def load_symbols_df() -> pd.DataFrame:
        """
        한국거래소 종목 목록 데이터프레임을 호출하는 메서드

        :return : 한국거래소 종목 목록 데이터프레임
        :rtype : pd.DataFrame
        """
        symbols_df = kq.symbol_stock()
        return symbols_df

    class SYMBOL_FILTER:
        """
        SYMBOl_FILTER : symbols를 filtering 하는 클래스
        """

        @staticmethod
        def filter__market(symbols_df: pd.DataFrame) -> pd.DataFrame:
            """
            market에 대한 필터링을 진행하는 메서드

            :param pd.DataFrame : symbols_df : 한국거래소 종목 목록 데이터프레임
            :return: MARKET이 [코스닥, 유가증권]에 속하는 데이터프레임
            :rtype: pd.DataFrame
            """
            filtered_symbols_df = symbols_df[
                (symbols_df["MARKET"].isin(["코스닥", "유가증권"]))
            ].copy()
            return filtered_symbols_df

        @staticmethod
        def filter__admin_issue(symbols_df: pd.DataFrame) -> pd.DataFrame:
            """
            ADMIN_ISSUE에 대한 필터링을 진행하는 메서드

            :param pd.DataFrame : symbols_df : 한국거래소 종목 목록 데이터프레임
            :return: ADMIN_ISSUE가 0인 데이터프레임
            :rtype: pd.DataFrame
            """
            filtered_symbols_df = symbols_df[(symbols_df["ADMIN_ISSUE"] == 0)].copy()
            return filtered_symbols_df

        @staticmethod
        def filter_sec_type(symbols_df: pd.DataFrame) -> pd.DataFrame:
            """
            SEC_TYPE에 대한 필터링을 진행하는 메서드

            :param pd.DataFrame : symbols_df : 한국거래소 종목 목록 데이터프레임
            :return: SEC_TYPE이 [ST, EF, EN]에 속하는 데이터프레임
            :rtype: pd.DataFrame
            """
            filtered_symbols_df = symbols_df[
                symbols_df["SEC_TYPE"].isin(["ST", "EF", "EN"])
            ].copy()
            return filtered_symbols_df

    def filter_symbols_df(self, symbols_df: pd.DataFrame) -> pd.DataFrame:
        """
        symbol_df 에 대한 필터링을 진행하는 메서드

        :param pd.DataFrame : symbols_df : 한국거래소 종목 목록 데이터프레임
        :return: SYMBOL_FILTER의 메서드를 거친 데이터프레임
        :rtype: pd.DataFrame
        """
        symbol_filter = self.SYMBOL_FILTER()
        filtered_symbols_df = symbol_filter.filter__market(symbols_df)
        filtered_symbols_df = symbol_filter.filter__admin_issue(filtered_symbols_df)
        filtered_symbols_df = symbol_filter.filter_sec_type(filtered_symbols_df)
        return filtered_symbols_df

    @staticmethod
    def get_symbols(symbols_df: pd.DataFrame) -> list:
        """
        symbols_df의 symbol을 중복을 제거하여 추출하는 메서드

        :param pd.DataFrame : symbols_df : 한국거래소 종목 목록 데이터프레임
        :return: symbols
        :rtype: list
        """
        symbols = sorted(set(symbols_df["SYMBOL"]))
        return symbols

    # SYMBOL_LOADER PIPELINE
    def __call__(self) -> list:
        """
        SYMBOL_LOADER의 파이프라인을 제공하는 메서드

        :return: 필터를 거친 symbols
        :rtype: list
        """
        symbols_df = self.load_symbols_df()
        filtered_symbols_df = self.filter_symbols_df(symbols_df)
        symbols = self.get_symbols(filtered_symbols_df)
        return symbols

In [5]:
class FUNDAMENTAL_LOADER:
    """
    FUNDAMENTAL_LOADER : fundamental_analysis를 위한 정보를 추출하는 클래스
    """

    def __init__(self, symbol: str, date: dt.date) -> None:
        """
        FUNDAMENTAL_LOADER의 생성자

        :param str symbol: stock의 symbol 입니다.
        :param datetime.date date: 현재 날짜 입니다.

        :attr : pd.DataFrmae daily_stock_df : 현재 날짜 기준 가장 최근 stock데이터 입니다.
        """
        self.symbol = symbol
        self.date = date
        self.daily_stock_df = kq.daily_stock(
            symbol,
            start_date=date - dt.timedelta(days=7),
            end_date=date,
        )

    def load_recent_close(self) -> float:
        """
        가장 최근 종가를 추출합니다.
        :return: 종가
        :rtype: float
        """
        daily_stock_df = self.daily_stock_df
        _close = daily_stock_df.sort_values("DATE").tail(1)["CLOSE"].values[0]
        return _close

    def load_recent_marketcap(self) -> float:
        """
        가장 최근 시가총액을 추출합니다.

        :return: 시가총액
        :rtype: float
        """
        daily_stock_df = self.daily_stock_df
        _marketcap = daily_stock_df.sort_values("DATE").tail(1)["MARKETCAP"].values[0]
        return float(_marketcap)

    def load_recent_netprofit(self) -> float:
        """
        공시자료 중 가장 최근 당기순이익을 추출합니다.

        :return: 당기순이익
        :rtype: float
        """
        netprofit_df = kq.account_history(self.symbol, "122700")
        netprofit_df.sort_values("YEARMONTH", inplace=True)
        _netprofit = netprofit_df.tail(1)["VALUE"].values[0] * 1000
        return float(_netprofit)

    def load_recent_capital(self) -> float:
        """
        공시자료 중 가장 최근 총자산(총자본 - 총부채)를 추출합니다.

        :return: 총 자산(총 자본 - 총 부채)
        :rtype: float
        """
        capital_df = kq.account_history(self.symbol, "115000")
        capital_df.sort_values("YEARMONTH", inplace=True)
        _capital = capital_df.tail(1)["VALUE"].values[0] * 1000
        return float(_capital)

    def __call__(self) -> dict:
        """
        fundmanetal analysis를 위해 필요한 데이터를 가져와서 dictionary를 반환한다.

        :return: fundamental analysis를 위한 데이터 dictionary
        :rtype: dict
        """
        _close = self.load_recent_close()
        _marketcap = self.load_recent_marketcap()
        _netprofit = self.load_recent_netprofit()
        _capital = self.load_recent_capital()
        return {
            "SYMBOL": self.symbol,
            "CLOSE": _close,
            "MARKETCAP": _marketcap,
            "NETPROFIT": _netprofit,
            "CAPITAL": _capital,
        }

In [6]:
class PBR_PROCESSOR:
    """
    PBR_PROCESSOR : PBR에 대한 정보를 SCORE로 정제하여 반환하는 클래스
    """
    def __init__(self, fundamental_df) -> None:
        self.fundamental_df = fundamental_df

    @staticmethod
    def append_pbr(fundamental_df):
        fundamental_df["PBR"] = fundamental_df["MARKETCAP"] / (
            fundamental_df["CAPITAL"]
        )
        return fundamental_df

    @staticmethod
    def filter_negative_pbr(fundamental_df):
        fundamental_df = fundamental_df[fundamental_df["PBR"] > 0]
        return fundamental_df

    @staticmethod
    def append_score(low_pbr_df: pd.DataFrame) -> pd.DataFrame:
        low_pbr_df["SCORE"] = low_pbr_df["PBR"].sum() / low_pbr_df["PBR"]
        return low_pbr_df

    def __call__(self):
        fundamental_df = self.fundamental_df

        fundamental_df = self.append_pbr(fundamental_df)
        fundamental_df = self.filter_negative_pbr(fundamental_df)
        fundamental_df = self.append_score(fundamental_df)

        score_df = fundamental_df.loc[:, ["SYMBOL", "SCORE", "CLOSE"]]
        return score_df

In [7]:
class BUYING_ORDER_PROCESSOR:
    """
    BUYING_ORDER_PROCESSOR : 매수주문을 생성하는 클래스
    """
    def __init__(self, score_df, invest_money, status_df, n) -> None:
        """
        BUYING_ORDER_PROCESSOR의 생성자
        
        :param pd.DataFrame score_df: symbol,score,close를 가진 데이터프레임
        :param float invest_money: 당일 활용 투자 금액
        :param pd.DataFrame status_df: 현재 position과 관련된 정보를 가진 데이터프레임
        :param int n: 당일 투자종목의 갯수
        """
        self.score_df = score_df
        self.invest_money = invest_money
        self.status_df = status_df
        self.n = n

    @staticmethod
    def filter_positioned_symbol(score_df, positioned_symbol):
        """
        이미 position이 있는 종목을 필터링 하는 메소드
        
        :param pd.DataFrame score_df: symbol,score,close를 가진 데이터프레임
        :param list positioned_symbol: 현재 이미 position이 있는 symbol의 리스트
        """
        filtered_score_df = score_df[~(score_df["SYMBOL"].isin(positioned_symbol))]
        return filtered_score_df

    @staticmethod
    def get_high_score_df(score_df, n):
        """
        score_df의 상위 n개의 row를 추출하는 메서드
        
        :param pd.DataFrame score_df: symbol,score,close를 가진 데이터프레임
        :param int n: 당일 투자종목의 갯수
        """
        high_score_df = score_df.nlargest(n, "SCORE")
        return high_score_df

    @staticmethod
    def append_price_invest(high_score_df, invest_money):
        """
        당일 활용 투자 금액을 score 기준으로 분배한 column을 생성하는 메서드
        
        :param pd.DataFrame high_score_df: score_df의 score 상위 데이터프레임
        :param float invest_money: 당일 활용 투자 금액
        """
        high_score_df["PRICE_INVEST"] = (
            high_score_df["SCORE"] / high_score_df["SCORE"].sum()
        ) * invest_money
        return high_score_df

    @staticmethod
    def append_cnt_invest(high_score_df: pd.DataFrame) -> pd.DataFrame:
        """
        당일 활용 투자 금액 최근 종가로 나누어 주문 갯수를 column으로 생성하는 메서드
        
        :param pd.DataFrame high_score_df: score_df의 score 상위 데이터프레임
        """
        high_score_df["CNT_INVEST"] = (
            high_score_df["PRICE_INVEST"] // high_score_df["CLOSE"]
        )
        return high_score_df

    @staticmethod
    def get_order_from_df(df):
        """
        데이터 프레임에서 signiture에 맞게 주문 list를 추출하는 메서드
        
        :param pd.DataFrame df: [SYMBOL,CNT_INVEST]를 가진 데이터프레임
        """
        orders = list(
            df.set_index("SYMBOL")["CNT_INVEST"].astype(int).to_dict().items()
        )
        return orders

    def __call__(self) -> list:
        """
        BUYING_ORDER_PROCESSOR의 pipeline을 진행하는 메서드
        
        """
        score_df = self.score_df
        invest_money = self.invest_money
        status_df = self.status_df
        n = self.n
        
        positioned_symbol = sorted(set(status_df["SYMBOL"]))
        filtered_positioned_df = self.filter_positioned_symbol(
            score_df, positioned_symbol
        )

        high_score_df = self.get_high_score_df(filtered_positioned_df, n)
        high_score_df = self.append_price_invest(high_score_df, invest_money)
        high_score_df = self.append_cnt_invest(high_score_df)
        buying_order = self.get_order_from_df(high_score_df)
        return buying_order

In [8]:
class SELLING_ORDER_PROCESSOR:
    """
    SELLING_ORDER_PROCESSOR : 매도주문을 추출하는 클래스
    """

    def __init__(self, score_df, status_df, percentage) -> None:
        """
        SELLING_ORDER_PROCESSOR의 생성자

        :param pd.DataFrame score_df: symbol,score,close를 가진 데이터프레임
        :param pd.DataFrame status_df: 현재 position과 관련된 정보를 가진 데이터프레임
        :param int percentage: low_score를 선정하기 위한 하위 (n-percent)의 기준
        """
        self.score_df = score_df
        self.status_df = status_df
        self.percentage = percentage

    @staticmethod
    def get_limit_line(score_df, percentage):
        """
        score 하위 [percentage]% 를 한계선으로 설정하고 이를 기준값을 추출

        :param pd.DataFrame score_df: symbol,score,close를 가진 데이터프레임
        :param int percentage: low_score를 선정하기 위한 하위 (n-percent)의 기준
        """
        limit_line = np.percentile(score_df["SCORE"], percentage)
        return limit_line

    @staticmethod
    def get_low_score_df(score_df: pd.DataFrame, limit_line: float) -> pd.DataFrame:
        """
        기준값을 바탕으로 low_score_df를 추출하는 메서드

        :param pd.DataFrame score_df: symbol,score,close를 가진 데이터프레임
        :param float limit_line: get_limit_line에서 추출한 하위 한계선
        """
        low_score_df = score_df[score_df["SCORE"] < limit_line]
        return low_score_df

    @staticmethod
    def get_filtered_position_symbols(
        low_score_df: pd.DataFrame, position_symbols: list
    ) -> pd.DataFrame:
        """
        low_score_df 중 position_symbol에 속하는 symbol만 추출하는 메서드

        :param pd.DataFrame low_score_df: symbol,score,close를 가진 데이터프레임
        :param list position_symbols: 현재 이미 position이 있는 symbol의 리스트
        """
        filtered_position_symbols = sorted(
            set(low_score_df["SYMBOL"]) & set(position_symbols)
        )
        return filtered_position_symbols

    @staticmethod
    def get_order_from_df(df):
        """
        데이터 프레임에서 signiture에 맞게 주문 list를 추출하는 메서드

        :param pd.DataFrame df: [SYMBOL,CURRENT_QTY]를 가진 데이터프레임
        """
        orders = list(
            df.set_index("SYMBOL")["CURRENT_QTY"]
            .apply(lambda x: x * -1)
            .astype(int)
            .to_dict()
            .items()
        )
        return orders

    def __call__(self) -> list:
        """
        SELLING_ORDER_PROCESSOR의 pipeline을 진행하는 메서드
        """
        score_df = self.score_df
        status_df = self.status_df
        percentage = self.percentage
        position_symbols = sorted(set(status_df["SYMBOL"]))

        limit_line = self.get_limit_line(score_df, percentage)
        low_score_df = self.get_low_score_df(score_df, limit_line)
        filtered_position_symbols = self.get_filtered_position_symbols(
            low_score_df, position_symbols
        )

        selling_df = status_df[status_df["SYMBOL"].isin(filtered_position_symbols)]
        selling_orders = self.get_order_from_df(selling_df)

        return selling_orders

In [15]:
status_loader = STATUS_LOADER(dict(), dict())

current_cash = status_loader.get_current_cash()
invest_money = current_cash / 1.5

status_df = status_loader.get_status_df()
position_symbols = sorted(set(status_df["SYMBOL"]))

In [16]:
symbol_loader = SYMBOL_LOADER()
total_symbols = symbol_loader()

sampled_symbols = random.sample(total_symbols, CFG["sampling_size"])
using_symbols = sorted(set(sampled_symbols + position_symbols))

In [19]:
from tqdm import tqdm

In [20]:
date = dt.date(2023, 8, 11)
fundamental_data_list = list()
for symbol in tqdm(using_symbols):
    try:
        _fundamental_loader = FUNDAMENTAL_LOADER(symbol, date)
        _fundamental_data = _fundamental_loader()
        fundamental_data_list.append(_fundamental_data)
    except:
        pass
fundamental_df = pd.DataFrame(fundamental_data_list)

100%|██████████| 250/250 [02:35<00:00,  1.61it/s]


In [21]:
pbr_processor = PBR_PROCESSOR(fundamental_df)
pbr_score_df = pbr_processor()

In [34]:
"""
자본 (자산 - 부채) : capital
자산 : asset
부채 : liability
"""

,SYMBOL,CLOSE,MARKETCAP,NETPROFIT,CAPITAL,PBR
0,000430,"6,100","378,200,000,000.0000","23,233,915,000.0000","598,011,882,000.0000",0.6324
1,000860,"21,250","138,125,000,000.0000","11,666,578,000.0000","678,845,782,000.0000",0.2035
2,000990,"54,200","2,406,403,469,600.0000","558,836,662,000.0000","1,638,284,078,000.0000",1.4689
3,001390,"8,570","594,162,213,600.0000","952,792,172,000.0000","3,240,866,490,000.0000",0.1833
4,001630,"54,200","271,534,466,200.0000","-22,663,808,000.0000","798,543,946,000.0000",0.3400
...,...,...,...,...,...,...
134,357230,"9,420","195,104,082,120.0000","14,460,551,000.0000","161,647,916,000.0000",1.2070
135,368770,"4,465","146,694,114,625.0000","-1,731,270,000.0000","29,123,290,000.0000",5.0370
136,383800,"7,890","601,854,644,100.0000","170,140,000,000.0000","1,611,093,000,000.0000",0.3736
137,408920,"2,920","124,545,168,600.0000","6,984,075,000.0000","44,049,530,000.0000",2.8274


In [31]:
buying_order_processor = BUYING_ORDER_PROCESSOR(score_df, 10_000_000, status_df, 5)
buying_orders = buying_order_processor()
selling_order_processor = SELLING_ORDER_PROCESSOR(score_df, status_df, 30)
selling_orders = selling_order_processor()

In [82]:
import random
import logging
import datetime as dt
import pandas as pd


def trade_func(
    date: dt.date,
    dict_df_result: dict[str, pd.DataFrame],
    dict_df_position: dict[str, pd.DataFrame],
    logger: logging.Logger,
) -> list[tuple[str, int]]:
    """
    STATUS_LOADER
    """
    status_loader = STATUS_LOADER(dict_df_result, dict_df_position)

    current_cash = status_loader.get_current_cash()
    invest_money = current_cash / 1.5

    status_df = status_loader.get_status_df()
    position_symbols = sorted(set(status_df["SYMBOL"]))

    """
    SYMBOL_LOADER
    """
    symbol_loader = SYMBOL_LOADER()
    total_symbols = symbol_loader()

    sampled_symbols = random.sample(total_symbols, CFG["sampling_size"])
    using_symbols = sorted(set(sampled_symbols + position_symbols))

    """
    FUNDAMENTAL_LOADER
    """
    fundamental_data_list = list()
    for symbol in using_symbols:
        try:
            _fundamental_loader = FUNDAMENTAL_LOADER(symbol, date)
            _fundamental_data = _fundamental_loader()
            fundamental_data_list.append(_fundamental_data)
        except:
            pass
    fundamental_df = pd.DataFrame(fundamental_data_list)

    """
    PBR_PROCESSOR
    """
    pbr_processor = PBR_PROCESSOR(fundamental_df)
    score_df = pbr_processor()

    """
    BUYING_ORDER_PROCESSOR
    """
    buying_order_processor = BUYING_ORDER_PROCESSOR(
        score_df, invest_money, status_df, CFG["buying_order_n"]
    )
    buying_orders = buying_order_processor()
    
    """
    SELLING_ORDER_PROCESSOR
    """
    selling_order_processor = SELLING_ORDER_PROCESSOR(
        score_df, status_df, CFG["selling_order_percentage"]
    )
    selling_orders = selling_order_processor()
    
    symbols_and_orders = buying_orders + selling_orders
    
    return symbols_and_orders

In [83]:
# loop
dict_df_result = kq.backtest_stock_port_daily(
    trade_func,
    "2023-08-11",  # 실제 심사에서는 투자기간 시작일
    "2023-09-11",  # 실제 심사에서는 투자기간 종료일
    init_cash=1_000_000_000,  # 10억원
)

[2023-08-11] 종목: 032190, 주문전 보유수량:      0 주문수량:  5,946, 매매수량:  5,946, 주문후 보유수량:  5,946
[2023-08-11] 종목: 052300, 주문전 보유수량:      0 주문수량: 230,352, 매매수량: 230,352, 주문후 보유수량: 230,352
[2023-08-11] 종목: 151860, 주문전 보유수량:      0 주문수량:  4,533, 매매수량:  4,533, 주문후 보유수량:  4,533
[2023-08-11] 종목: 084690, 주문전 보유수량:      0 주문수량:  7,497, 매매수량:  7,497, 주문후 보유수량:  7,497
[2023-08-11] 종목: 000210, 주문전 보유수량:      0 주문수량:  1,128, 매매수량:  1,128, 주문후 보유수량:  1,128
[2023-08-11] 종목: 004150, 주문전 보유수량:      0 주문수량: 14,900, 매매수량: 14,900, 주문후 보유수량: 14,900
[2023-08-11] 종목: 034810, 주문전 보유수량:      0 주문수량:  4,864, 매매수량:  4,864, 주문후 보유수량:  4,864
[2023-08-11] 종목: 002920, 주문전 보유수량:      0 주문수량: 12,896, 매매수량: 12,896, 주문후 보유수량: 12,896
[2023-08-11] 종목: 057050, 주문전 보유수량:      0 주문수량:    745, 매매수량:    745, 주문후 보유수량:    745
[2023-08-11] 종목: 035760, 주문전 보유수량:      0 주문수량:    430, 매매수량:    430, 주문후 보유수량:    430
[2023-08-14] 종목: 005440, 주문전 보유수량:      0 주문수량: 10,924, 매매수량: 10,924, 주문후 보유수량: 10,924
[2023-08-14] 종목: 021820, 주문전 보유수량:      

In [84]:
dict_df_result['TOTAL']['TOTAL_VALUE'].tail(1)

20    1,006,858,610
Name: TOTAL_VALUE, dtype: int64